# data-types-what-they-are-and-how-to-publish-them

## Setup

In [48]:
import pandas as pd
from datetime import date
import json
from rdflib import Graph, Namespace, URIRef, Literal, FOAF, XSD

## Create sample data

In [49]:
df=pd.DataFrame(data=[{'name':'Jane','age':40,'date_of_birth':date(1982,1,1),'homepage':'www.example.com/jane'},
                      {'name':'John','age':41,'date_of_birth':date(1981,1,1),'homepage':'www.example.com/john'}
                     ])
df.index.name='id'
df

,name,age,date_of_birth,homepage
id,,,,
0,Jane,40,1982-01-01,www.example.com/jane
1,John,41,1981-01-01,www.example.com/john


## Create markdown version (for blog post)

In [50]:
print(df.to_markdown(colalign=["center"]*5))

|  id  |  name  |  age  |  date_of_birth  |       homepage       |
|:----:|:------:|:-----:|:---------------:|:--------------------:|
|  0   |  Jane  |  40   |   1982-01-01    | www.example.com/jane |
|  1   |  John  |  41   |   1981-01-01    | www.example.com/john |


## Create csv version

In [51]:
df.to_csv('data.csv')
for line in open('data.csv'): print(line, end=' ')

id,name,age,date_of_birth,homepage
 0,Jane,40,1982-01-01,www.example.com/jane
 1,John,41,1981-01-01,www.example.com/john
 

## Create json version

In [52]:
with open('data.json', 'w') as f:
    json.dump(df.T.to_dict(),
              f, 
              indent=4, 
              sort_keys=True, 
              default=str  # used to convert the date to a string (otherwise an error is raised)
             )
for line in open('data.json'): print(line, end=' ')

{
     "0": {
         "age": 40,
         "date_of_birth": "1982-01-01",
         "homepage": "www.example.com/jane",
         "name": "Jane"
     },
     "1": {
         "age": 41,
         "date_of_birth": "1981-01-01",
         "homepage": "www.example.com/john",
         "name": "John"
     }
 } 

## Create xml version

In [53]:
with open('data.xml','w') as f:
    f.write(df.to_xml())
for line in open('data.xml'): print(line, end=' ')

<?xml version='1.0' encoding='utf-8'?>
 <data>
   <row>
     <id>0</id>
     <name>Jane</name>
     <age>40</age>
     <date_of_birth>1982-01-01</date_of_birth>
     <homepage>www.example.com/jane</homepage>
   </row>
   <row>
     <id>1</id>
     <name>John</name>
     <age>41</age>
     <date_of_birth>1981-01-01</date_of_birth>
     <homepage>www.example.com/john</homepage>
   </row>
 </data> 

## Create rdf version

In [56]:
g=Graph()
EX=Namespace('http://www.example.com/')
DBO=Namespace('http://dbpedia.org/ontology/')
g.bind('ex',EX)
g.bind('dbo',DBO)
for index,row in df.iterrows():
    resource=getattr(EX,str(index))
    g.add((resource,FOAF.name,Literal(row['name'])))
    g.add((resource,FOAF.age,Literal(row['age'])))
    g.add((resource,DBO.birthDate,Literal(row['date_of_birth'])))
    g.add((resource,FOAF.homepage,Literal(row['homepage'],datatype=XSD.anyURI)))
g.serialize('data.ttl',format='turtle')
for line in open('data.ttl'): print(line, end=' ')

@prefix dbo: <http://dbpedia.org/ontology/> .
 @prefix ex: <http://www.example.com/> .
 @prefix foaf: <http://xmlns.com/foaf/0.1/> .
 @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
 
 ex:0 dbo:birthDate "1982-01-01"^^xsd:date ;
     foaf:age 40 ;
     foaf:homepage "www.example.com/jane"^^xsd:anyURI ;
     foaf:name "Jane" .
 
 ex:1 dbo:birthDate "1981-01-01"^^xsd:date ;
     foaf:age 41 ;
     foaf:homepage "www.example.com/john"^^xsd:anyURI ;
     foaf:name "John" .
 
 